## A Model Builder

To read in atomic coordinates for molecules, orient them and prepare model input files (.xyz coordinate file, COLLECTIVE.include COLVAR.include CP2K Restrain files)

v2: add COM to save_xyz method

(add dihedral angles measurement tools:
input: atoms list[[a1,a2,a3,a4],[a5,a6,a7,a8],....], 
maybe also the COLVAR numbers of COM constrain --> so the dihedral can be added after them 

output: angle list[angle1,angle2,...]

(add dihedral constrain restrain writter for CP2K...



In [1]:
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
%matplotlib inline  
# maybe some 3D plot for the molecule?? Is it possible?
  

In [17]:
class Molecule:
    
    def __init__(self,Name,path_):
        self.Name = Name 
        self.path_ = path_
        
        #try parse path_ to get rid of file name
        #ugly, need polishment!
        self.directory = ''
        for x in path_.split('/')[1:-1]:
            self.directory = self.directory + '/' + x
        self.directory = self.directory + '/'
        
        with open (path_) as file_:
            self.coord_ = []
            self.atoms_ = []
            #self.Natom_ = file_.readlines()[0]            
            lines_ = file_.readlines()
            self.Natom_ = lines_[0]
            lines_ = lines_[2:int(self.Natom_)+2] #int(self.Natom_)+2 should be more general than -1 or something
            for line in lines_:
                split_ = [ float(x) for x in line.split()[1:] ]
                self.coord_.append( split_ )
                self.atoms_.append( line.split()[0] )
        #print (self.atoms_)
        self.center = []
        self.coord_c = []
        self.com_ = []
        self.combined_Natom_ = self.Natom_
        self.combined_atoms_ = self.atoms_
        self.combined_coord_ = []
           
    def print_coord(self):
        print (self.coord_)
        
    def calc_center(self):  # calc_center for whole molecule
        if self.coord_c:
            coord_t = np.array(self.coord_c).transpose()
        else:
            coord_t = np.array(self.coord_).transpose()
        # print (coord_t)
        x_ = np.mean( [ float(x) for x in coord_t[0] ] )
        y_ = np.mean( [ float(x) for x in coord_t[1] ] )
        z_ = np.mean( [ float(x) for x in coord_t[2] ] )
        self.center = [x_, y_, z_]
        print (self.center)
        
            # python does not support function overloading. 
            # given a atom_list [i,j,k,...], return the center for (atom i, atom j, atom k, and ...)
    def calc_center_atom(self, atom_list):      
        coord_a =[]
        for i in atom_list:
            if self.coord_c:
                coord_a.append( self.coord_c[i-1] )
            else:
                coord_a.append( self.coord_[i-1] )
        coord_a_t =np.array(coord_a).transpose()
        x_ = np.mean( [ float(x) for x in coord_a_t[0] ] )
        y_ = np.mean( [ float(x) for x in coord_a_t[1] ] )
        z_ = np.mean( [ float(x) for x in coord_a_t[2] ] )
        print (x_, y_, z_)
        
    def calc_COM(self):
        if self.coord_c:
            coord_t = np.array(self.coord_c).transpose()
        else:
            coord_t = np.array(self.coord_).transpose()
        #print(coord_t[2])
        
        dict_masses = {
            "H" : 1.00794,
            "C" : 12.0107,
            "CP" : 12.0107,
            "N"  : 14.0067,
            "O"  : 15.999,
            "S" : 32.065,
            "F" : 18.9984
        }
        
        mass_t = 0.0
        com_x = 0.0
        com_y = 0.0
        com_z = 0.0
        for i in range(len(coord_t[0])):
            m_i = dict_masses[self.atoms_[i]]
            #print(m_i)
            mass_t += m_i
            com_x += m_i*coord_t[0][i]
            com_y += m_i*coord_t[1][i]
            com_z += m_i*coord_t[2][i]
        self.com_ = [com_x/mass_t, com_y/mass_t, com_z/mass_t]
        print (self.com_)
        
    def calc_COM_atom(self, atom_list):
        coord_a =[]
        #self.combined_atoms_ = self.atoms_
        dict_masses = {
            "H" : 1.00794,
            "C" : 12.0107,
            "CP" : 12.0107,
            "N"  : 14.0067,
            "O"  : 15.999,
            "S" : 32.065,
            "F" : 18.9984
        }
        mass_ = []
        for i in atom_list:
            if self.coord_c:
                coord_a.append( self.coord_c[i-1] )
                mass_.append( dict_masses[self.atoms_[i-1]])
            else:
                coord_a.append( self.coord_[i-1] )
                mass_.append( dict_masses[self.atoms_[i-1]])
        
        coord_a_t =np.array(coord_a).transpose()
        #print(coord_a_t)
        
        mass_t = 0.0
        com_x = 0.0
        com_y = 0.0
        com_z = 0.0
        for i in range(len(coord_a_t[0])):
            m_i = mass_[i]
            #print(m_i)
            mass_t += m_i
            com_x += m_i*coord_a_t[0][i]
            com_y += m_i*coord_a_t[1][i]
            com_z += m_i*coord_a_t[2][i]
            
        return (com_x/mass_t, com_y/mass_t, com_z/mass_t)
                    
    def orien_center(self):  # move center to (0.0 0.0 0.0)
        #first check if already centered
        if not self.center:
            self.calc_center()
        
        # try polish here!!
        k = 0
        while (self.center[k]<10**-6):
            self.center[k] = 0.0
            k = k+1
            if (k>2):
                break

        for i in range(len(self.coord_)):
            self.coord_c.append( [x1-x2 for (x1,x2) in zip(self.coord_[i],self.center)] )
        #print( self.coord_c)
        #try deal with the degits of self.coord_c
        
    def orien_COM(self):  # move center to (0.0 0.0 0.0)
        #first check if already centered
        if not self.com_:
            self.calc_COM()
        
        # try polish here!!
        k = 0
        while (self.com_[k]<10**-6):
            self.com_[k] = 0.0
            k = k+1
            if (k>2):
                break

        for i in range(len(self.coord_)):
            self.coord_c.append( [x1-x2 for (x1,x2) in zip(self.coord_[i],self.com_)] )
        #print( self.coord_c)
        #try deal with the degits of self.coord_c

    def image_xz(self):
        #first check if centered (in the future, maybe also check if align long-short axis)
        if not self.coord_c:
            self.orien_center()
        #print (len(self.coord_))
        for i in range( len(self.coord_) ):
            #print (self.coord_c[i][1])
            self.coord_c[i][1] = -1*self.coord_c[i][1]
        #print( self.coord_c)
    
    def shift_z(self, dist):
        if not self.coord_c:
            #self.orien_center()
            self.coord_c = self.coord_
        for i in range( len(self.coord_) ):
            self.coord_c[i][2] = self.coord_c[i][2] + dist
        #print( self.coord_c)        
        
    def combine_molecule(self, obj):
        #print ( obj.Name )
        self.combined_Natom_ = int(self.combined_Natom_) + int(obj.Natom_)
        print (self.combined_Natom_)
        if not self.center:
            self.calc_center()
        if not obj.center:
            obj.calc_center()
        self.combined_atoms_ = self.combined_atoms_ + obj.atoms_
        if not self.combined_coord_:
            self.combined_coord_ = self.coord_c + obj.coord_c
        else:
            self.combined_coord_ = self.combined_coord_ + obj.coord_c
        #print(self.combined_atoms_)
        #print('length:',len(self.combined_atoms_))
        #print(self.combined_coord_)

    #DA: dihedral angle
    def measure_DA(self,atom_list_list):
        
        for i in atom_list_list:
            
            v1 = [x1-x2 for (x1,x2) in zip(self.coord(atom_list_list[i][2]),self.coord(atom_list_list[i][1]))]
            v2 = [x1-x2 for (x1,x2) in zip(self.coord(atom_list_list[i][3]),self.coord(atom_list_list[i][2]))]   
            v3 = [x1-x2 for (x1,x2) in zip(self.coord(atom_list_list[i][4]),self.coord(atom_list_list[i][3]))]
            
            #normal vectors:
            n1 = [v1[1]v2[2]-v1[2]v2[1],v1[2]v2[0]-v1[0]v2[2],v1[0]v2[1]-v1[1]v2[0]]
            n2 = [v2[1]v3[2]-v2[2]v3[1],v2[2]v3[0]-v2[0]v3[2],v2[0]v3[1]-v2[1]v3[0]]
            n1_l = (n1[0]**2 + n1[1]**2 + n1[2]**2)**0.5
            n2_l = (n2[0]**2 + n2[1]**2 + n2[2]**2)**0.5
            n1 = [x/n1_l for x in n1]
            n2 = [x/n2_l for x in n2]
            
            theta = acos(n1[0]n2[0]+n1[1]n2[1]+n1[2]n2[2])
            print(theta)
            
        
    def save_xyz(self, option):
        if not self.coord_c:
            #self.orien_center()
            self.coord_c = self.coord_
        if option == 'imaged':
            self.image_xz()
        if option == 'combined':
            Natom_ = str(self.combined_Natom_) + '\n'
            atoms_ = self.combined_atoms_

        else:
            Natom_ = self.Natom_
            atoms_ = self.atoms_
        coord_ = {
            'default': self.coord_ ,
            'centered': self.coord_c ,
            'shifted': self.coord_c ,
            'imaged': self.coord_c,
            'combined': self.combined_coord_ ,
            'COM': self.coord_c,         # add in the future
        }[option]
        
        #print (coord_)
        
        #write files
        path_w = self.directory + self.Name + '-' + option + '.xyz'
        with open (path_w ,'w') as fout_:
            fout_.write( Natom_ )
            fout_.write('\n')
            for i in range(len(atoms_)):
                fout_.write(atoms_[i] +' ' + str(coord_[i][0]) + ' ' + str(coord_[i][1]) + ' ' + str(coord_[i][2]) + '\n')        
                

    #for COLVAR and COLLECTIVE include files    
    def cp2k_restrain_com(self, atom_list_list):
        
        path_colvar = self.directory + 'COLVAR.include'
        path_collective = self.directory + 'COLLECTIVE.include'
        count = 0
        print(len(atom_list_list))
        for i in range(len(atom_list_list)):
            
            if (count ==0):
                ww = "w"
            else:
                ww = "a"
            
            with open(path_collective, ww) as out:
                count += 1
                list_ = " ".join(map(str ,atom_list_list[i]))
                #list_com = " ".join(map(str ,['{:.8f}'.format(1.8897161646*x) for x in self.calc_COM_atom(atom_list_list[i])]))
                list_com = " ".join(['{:.8f}'.format(1.8897161646*x) for x in self.calc_COM_atom(atom_list_list[i])])
                
                out.write("""
   &COLLECTIVE
           COLVAR %s
           TARGET [angstrom] 0.0
           &RESTRAINT
                   K 0.005
           &END RESTRAINT
           INTERMOLECULAR
   &END COLLECTIVE\n\n
   """ %(count))
           
        #here write COLVAR.include
            with open(path_colvar, ww) as out2:
                out2.write("""
   &COLVAR
      &DISTANCE
           &POINT
                ATOMS %s
                TYPE GEO_CENTER
           &END POINT
           &POINT
               XYZ %s
               TYPE FIX_POINT
           &END POINT
           ATOMS 1 2
      &END DISTANCE 
   &END COLVAR\n
   """ %(list_, list_com))
   
        

    def cp2k_restrain_dihedral(self, atom_list_list,COLVAR_COM):
        
        path_colvar = self.directory + 'COLVAR.include-DAngle'
        path_collective = self.directory + 'COLLECTIVE.include-DAngle'
        count = COLVAR_COM
        print(len(atom_list_list))
        for i in range(len(atom_list_list)):
            
            if (count == COLVAR_COM):
                ww = "w"
            else:
                ww = "a"
            
            with open(path_collective, ww) as out:
                count += 1
                list_ = " ".join(map(str ,atom_list_list[i]))
                #list_com = " ".join(map(str ,['{:.8f}'.format(1.8897161646*x) for x in self.calc_COM_atom(atom_list_list[i])]))
                list_com = " ".join(['{:.8f}'.format(1.8897161646*x) for x in self.calc_COM_atom(atom_list_list[i])])
                
                #Maybe 180.0 and 0.005 should be set as variables
                out.write("""
   &COLLECTIVE
           COLVAR %s
           TARGET [deg] 180.0 
           &RESTRAINT
                   K 0.005
           &END RESTRAINT
           INTERMOLECULAR
   &END COLLECTIVE\n\n
   """ %(count))
           
        #here write COLVAR.include
            with open(path_colvar, ww) as out2:
                out2.write("""
   &COLVAR
      &TORSION
          ATOMS %s
      &END TORSION
   &END COLVAR\n
   """ %(list_))
        
    #def cp2k_
        
    #def orien_x(self):  # from center to an assigned point as x-axis 
        
    #def orien_y(self):  # rotate along x-axis, this seems tricky!
        
    #def rot_aboutZ(self, degree):
        
    #def rot_aboutXYZ(self, axis, degree):
    
    #def calc_avg_plane(self):
    
    #def calc_long_short_axis(self):
        #first check if already calc_avg_plane

In [26]:
path_C60_13OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-OT9/C60-13OT9.xyz'
OT9_13 = Molecule("OT9_13", path_C60_13OT9)
#OT9_13.calc_COM()
#list_1 = [list(range(1,61)),list(range(61,126))]
#print(list_)
constraint_num = 29
list_1 = [[] for x in range(constraint_num)]


#print(list_1)
                 
for i in range(12):
    #print(i)
    list_1[i] = list(range(60+8+i*65, 60+8+i*65+29))

for j in range(12,24):
    list_1[j] = list(range(60+37+(j-13)*65, 60+37+(j-13)*65+29))
#print(list_1)


list_1[24] = list(range(841, 848))
list_1[25] = list(range(869, 877))
list_1[26] = list(range(898, 906))

list_1[27] = list(range(1,61))

list_1[0] = list(range(61,68))
list_1[12] = list(range(89,97))
list_1[28] = list(range(118,126))

OT9_13.cp2k_restrain(list_1)


29


In [22]:
path_C60_13OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-OT9/C60-13OT-NVT-raiseTemperature/OT9_13-COM.xyz'
OT9_13 = Molecule("OT9_13", path_C60_13OT9)
#OT9_13.calc_COM()
#list_1 = [list(range(1,61)),list(range(61,126))]
#print(list_)
constraint_num = 39
list_1 = [[] for x in range(constraint_num)]
#list_1[0] = list(range(1,61))

#print(list_1)
                 
for i in range(13):
    #print(i)
    list_1[i] = list(range(22+i*65, 22+i*65+22))

for j in range(13,26):
    list_1[j] = list(range(44+(j-13)*65, 44+(j-13)*65+22))
#print(list_1)

for k in range(26,39):
    list_1[k] = list(range(1+(k-26)*65, 1+(k-26)*65+65))

OT9_13.cp2k_restrain(list_1)



39


In [8]:
path_C60_13OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-OT9/C60-13OT-NVT-raiseTemperature/Only-13OT9-init.xyz'
OT9_13 = Molecule("OT9_13", path_C60_13OT9)
#OT9_13.calc_COM()
#list_1 = [list(range(1,61)),list(range(61,126))]
#print(list_)
constraint_num = 13
list_1 = [[] for x in range(constraint_num)]
#list_1[0] = list(range(1,61))

#print(list_1)
                 
for i in range(13):
    #print(i)
    list_1[i] = list(range(1+i*65, 1+i*65+65))


OT9_13.cp2k_restrain(list_1)

13


In [19]:
#XYZ -4.19751264207e-17 0.0447882456407 13.227921027
path_C60_13OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-OT9/C60-13OT-NVT-raiseTemperature/Only-13OT9-init.xyz'
OT9_13 = Molecule("OT9_13", path_C60_13OT9)
#OT9_13.calc_COM()
#list_1 = [list(range(1,61)),list(range(61,126))]
#print(list_)
#constraint_num = 1
#list_1 = [[] for x in range(constraint_num)]
#list_1[0] = list(range(1,61))

#print(list_1)
                 

#list_1[0] = list(range(1,6))


#OT9_13.cp2k_restrain(list_1)
#OT9_13.calc_center_atom(list_1[0])
OT9_13.orien_COM()
OT9_13.save_xyz('COM')


[-1.4845945109644216e-08, 0.0068334052004017046, 29.800017781288087]


In [ ]:
class cp2k_input:
    
    def __init__(self,path):
        self.path = path
        self.sites =       #how many sites: diabatic states
        
    def frz_couplings(self):
        path_w = self.path + 'FRZ_COUPLINGS.include'
        count = 2*2 + 3*(self.sites-2) + lamdas    #total numbers of non-zero elements in tight-binding Hamiltonian
        with open (path_w, 'w') as out:
            out.write('CONNECTIVITY_FRZ_C    T\n')
            out.write('CONNECTIVITY_FRZ_C     %d\n' % count)
            #Not finish yet....
            
    def ff_active(self):
        path_w = self.path + 'FF_ACTIVE.include'
        with open (path_w, 'w') as out:
            for i in range(self.sites):
                out.write('@SET  SITE_ENERGY %d\n' % i+1)
                out.write(' @INCLUDE ./FORCE_EVAL.include')
    
    def site_energies(self):
        path_w = self.path + '/site_energies'
            
            
    
        

In [139]:
#Writting Dummy VELOC.init
with open ('/Users/louispeng/Documents/Jochen/Meeting/Geom/VELOC.init','w') as out:
    for i in range(905):
        out.write('    0.0    0.0    0.0')
        out.write('\n')

In [58]:
#Testing .join function with map.... Q_what are these functions???
a = [1,2,3,4]
b = " ".join(map(str,a))
print(b)


1 2 3 4


In [134]:
path_C60_13OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-13OT9.xyz'
OT9_13 = Molecule("OT9_13", path_C60_13OT9)
#OT9_13.calc_COM()
#list_1 = [list(range(1,61)),list(range(61,126))]
#print(list_)
constraint_num = 14
list_1 = [[] for x in range(constraint_num)]
list_1[0] = list(range(1,61))

#print(list_1)
                 
for i in range(13):
    #print(i)
    list_1[i+1] = list(range(61+i*65, 61+i*65+65))
        
#print(list_1)

OT9_13.cp2k_restrain(list_1)

In [355]:
path_C60_OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60_OT9-shifted.xyz'
C60_OT9 = Molecule('C60_OT9', path_C60_OT9)
#C60_center = [1, 2, 4, 6, 8, 38, 31, 32, 34, 36]

unit = [1, 2, 3, 4, 6]
OT9_center1 = []
OT9_center2 = []
for i in range(9):
    num1 = 60+ (i*65) + 28
    num2 = 60+ (i*65) + 57
    unit1 = []
    unit2 = []
    for j in unit:
        unit1.append(num1+j)
        unit2.append(num2+j)
    print(unit1)
    print(unit2)
    OT9_center1.append(unit1)
    OT9_center2.append(unit2)

for k in range(len(OT9_center1)):
    C60_OT9.calc_center_atom(OT9_center1[k])
    C60_OT9.calc_center_atom(OT9_center2[k])


#C60_OT9.calc_center_atom(C60_center)
#C60_OT9.shift_z(7.0)
#C60_OT9.calc_center_atom(C60_center)
# C60_OT9.calc_center()
# print (C60_OT9.Natom_)
#C60_OT9.save_xyz('shifted')


[89, 90, 91, 92, 94]
[118, 119, 120, 121, 123]
[154, 155, 156, 157, 159]
[183, 184, 185, 186, 188]
[219, 220, 221, 222, 224]
[248, 249, 250, 251, 253]
[284, 285, 286, 287, 289]
[313, 314, 315, 316, 318]
[349, 350, 351, 352, 354]
[378, 379, 380, 381, 383]
[414, 415, 416, 417, 419]
[443, 444, 445, 446, 448]
[479, 480, 481, 482, 484]
[508, 509, 510, 511, 513]
[544, 545, 546, 547, 549]
[573, 574, 575, 576, 578]
[609, 610, 611, 612, 614]
[638, 639, 640, 641, 643]
15.691537 -0.348744892309 6.99983736923
-15.691537 -0.348744692309 7.00043516923
15.691537 0.348744892309 10.7998373692
-15.691537 0.348744692309 10.8004351692
15.691537 -0.348744892309 14.5998373692
-15.691537 -0.348744692309 14.6004351692
15.691537 0.348744892309 18.3998373692
-15.691537 0.348744692309 18.4004351692
15.691537 -0.348744892309 22.1998373692
-15.691537 -0.348744692309 22.2004351692
15.691537 0.348744892309 25.9998373692
-15.691537 0.348744692309 26.0004351692
15.691537 -0.348744892309 29.7998373692
-15.691537 -0.348

In [304]:
OT9_11.shift_z(11.4)
OT9_.combine_molecule(OT9_11)
OT9_21.image_xz()
OT9_21.shift_z(15.2)
OT9_.combine_molecule(OT9_21)
OT9_12.shift_z(19.0)
OT9_.combine_molecule(OT9_12)
OT9_22.image_xz()
OT9_22.shift_z(22.8)
OT9_.combine_molecule(OT9_22)
OT9_13.shift_z(26.6)
OT9_.combine_molecule(OT9_13)
OT9_23.image_xz()
OT9_23.shift_z(30.4)
OT9_.combine_molecule(OT9_23)
OT9_.save_xyz('combined')

[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
320
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
385
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
450
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
515
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
580
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
645


In [303]:
path_OT9 = '/Users/louispeng/Documents/Jochen/Meeting/Geom/OT9_b3_n.xyz'
OT9_ = Molecule('OT9', path_OT9)
OT9_1 = Molecule('OT9', path_OT9)
OT9_2 = Molecule('OT9', path_OT9)
OT9_11 = Molecule('OT9', path_OT9)
OT9_21 = Molecule('OT9', path_OT9)
OT9_12 = Molecule('OT9', path_OT9)
OT9_22 = Molecule('OT9', path_OT9)
OT9_13 = Molecule('OT9', path_OT9)
OT9_23 = Molecule('OT9', path_OT9)
#OT9_.print_coord()
#print (OT9_.Natom_)
OT9_.orien_center()
OT9_.image_xz()
#print (OT9_.directory)
#OT9_.save_xyz('centered')
#OT9_.save_xyz('imaged')
OT9_1.shift_z(3.8)
OT9_.combine_molecule(OT9_1)
OT9_.combine_molecule(C60_)
OT9_2.image_xz()
OT9_2.shift_z(7.6)
OT9_.combine_molecule(OT9_2)

#for i in range(3):  #why this doesn't work???
#OT9_1.shift_z(7.6)
#OT9_.combine_molecule(OT9_1)
#OT9_2.shift_z(7.6)
#OT9_.combine_molecule(OT9_2)

#OT9_.save_xyz('combined')

[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
130
190
[5.465713352000771e-17, 0.08330150769230768, -1.6769230769230776e-05]
255


In [317]:
path_ = '/Users/louispeng/Documents/Jochen/Meeting/Geom/C60-neutral-B3opt.xyz'
#xyz = []  # xyz should be defined before put into constructor!
C60_ = Molecule('C60', path_)
C60_.calc_center()
#C60_.print_coord()
#C60_.orien_center()
#C60_.print_coord()
#C60_.orien_center()
#print (C60_.Natom_)
#C60_.shift_z(-7.)

[-7.4014868308343768e-18, 3.333333335279557e-08, 5.0000000031043729e-08]
